In [1]:
import numpy as np
from model import *

In [2]:
config = {}
config['epochs'] = 2
config['temperature'] = 1
config['num_neurons'] = 100
config['num_layers'] = 1
config['input_dim'] = None
config['learning_rate'] = 0.001
config['saved_path'] = 'Saved_weights.pth'


In [3]:
# Check if your system supports CUDA
use_cuda = torch.cuda.is_available()

# Setup GPU optimization if CUDA is supported
if use_cuda:
    computing_device = torch.device("cuda")
    extras = {"num_workers": 1, "pin_memory": True}
    print("CUDA is supported")
else: # Otherwise, train on the CPU
    computing_device = torch.device("cpu")
    extras = False
    print("CUDA NOT supported")

CUDA is supported


In [4]:
train_file = open('train.txt', 'r')
train_file = train_file.read()
train_songs = []

val_file = open('val.txt', 'r')
val_file = val_file.read()
val_songs = []

test_file = open('test.txt', 'r')
test_file = test_file.read()
test_songs = []

#split the songs into their own strings, while including start and end tags
for song in train_file.split('<start>')[1:]:
    #remove \n after <end> tag
    train_songs.append('<start>' + song[:-1])
    
for song in val_file.split('<start>')[1:]:
    #remove \n after <end> tag
    val_songs.append('<start>' + song[:-1])

for song in test_file.split('<start>')[1:]:
    #remove \n after <end> tag
    test_songs.append('<start>' + song[:-1])
    
    
unique_characters = list(set(train_file))
config['input_dim'] = len(unique_characters)

#create one hot encodings for each unique character in the alphabet of the training data
one_hot_dict_encode = {}
one_hot_dict_decode = {}
index = 0
for unique_character in unique_characters:
    current_encoding = np.zeros(len(unique_characters))
    current_encoding[index] = 1
    
    one_hot_dict_encode[unique_character] = current_encoding
    one_hot_dict_decode[tuple(current_encoding)] = unique_character
    
    index += 1


print(unique_characters)

[':', 'e', '\t', 'T', ',', '(', 'z', 'k', 'i', 's', '{', ')', 'O', '"', 't', 'n', 'R', '}', 'D', 'X', 'u', 'I', 'P', 'm', 'y', 'o', 'L', '!', '?', 'Z', '/', 'b', 'j', '8', '~', 'w', '_', 'q', '0', 'p', 'H', '^', 'a', '9', '|', '.', '\\', 'f', 'Y', '-', 'v', 'K', 'B', 'd', 'c', 'l', 'r', "'", '&', 'C', '7', 'S', 'F', '5', 'W', '4', 'E', 'g', '#', 'M', '>', '*', 'N', ']', 'J', '2', '+', '@', 'x', '<', ' ', 'h', 'U', '3', '\n', '=', 'V', 'Q', '6', '1', 'A', '[', 'G']


In [5]:
hidden_state = torch.zeros((config['num_layers'], 1, config['input_dim'])).to(computing_device)
cell_state = torch.zeros((config['num_layers'], 1, config['input_dim'])).to(computing_device)
hidden = (hidden_state, cell_state)

model = Lstm(config['input_dim'], config['num_neurons'], config['num_layers'], hidden)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=config['learning_rate'])

In [13]:
for i in range(config['epochs']):
    #TODO : randomize the songs here
    
    
    #train
    model.train()
    for song in train_songs:
#         checkpoint = torch.load(config['saved_path'])

        #set states to 0 at the beginning of each song
        hidden_state = torch.zeros((config['num_layers'], 1, config['input_dim'])).to(computing_device)
        cell_state = torch.zeros((config['num_layers'], 1, config['input_dim'])).to(computing_device)
        hidden = (hidden_state, cell_state)

        #restore the model's weights 
#         model.load_state_dict(checkpoint['model_state_dict'])
#         optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

        #train here
        model.zero_grad()
        
        #encode the characters to their respective one hot encoding
        encoded_inputs = torch.from_numpy(np.array([one_hot_dict_encode[character] for character in song]))         
        encoded_labels = torch.from_numpy(np.array(encoded_inputs[1:]))
        
        #train
        for i in range(len(encoded_inputs)):
            print(encoded_inputs[i].to(computing_device))
            print(hidden[0].shape, hidden[1].shape)
            output, hidden = model(encoded_inputs[i].to(computing_device), hidden)
            print('output')
            print(output, output.shape)
            print('squeezed')
            print(output.squeeze, output.squeeze().shape)
            loss += criterion(output.squeeze(), encoded_labels[i].to(computing_device))
            print(loss, type(loss))

            #every 100 characters, backprop
            if i % 99 == 0 or i == len(encoded_inputs) - 1:
                print('Training loss for minibatch ' + str(i/99) + ' : ' + str(loss.item()))
                loss.backward()
                optimizer.step()
                loss = 0
        
        
        #save model
#         torch.save({
#                 'model_state_dict': net.state_dict(),
#                 'optimizer_state_dict': optimizer.state_dict(),
#             }, config['saved_path'])
        
    #validation
    model.eval()
    with torch.no_grad():
        for song in val_songs:
            
            #set states to 0 at the beginning of each song
            hidden_state = torch.zeros((config['num_layers'], 1, config['input_dim'])).to(computing_device)
            cell_state = torch.zeros((config['num_layers'], 1, config['input_dim'])).to(computing_device)
            hidden = (hidden_state, cell_state)
            
            
        

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.], device='cuda:0', dtype=torch.float64)
torch.Size([1, 1, 93]) torch.Size([1, 1, 93])


TypeError: forward() takes from 2 to 3 positional arguments but 4 were given